In [13]:
import os
import logging
import mysql.connector
from mysql.connector import Error
from datetime import datetime
import torch
from diffusers import DiffusionPipeline
from diffusers.models.lora import LoRACompatibleConv
from celery import Celery
import json
from typing import Optional  # <-- Add this import
from PIL import Image

# 모델 이름
model_name = "stabilityai/sdxl-turbo"

def seamless_tiling(pipeline, x_axis, y_axis):
    def asymmetric_conv2d_convforward(self, input: torch.Tensor, weight: torch.Tensor, bias: Optional[torch.Tensor] = None):
        self.paddingX = (self._reversed_padding_repeated_twice[0], self._reversed_padding_repeated_twice[1], 0, 0)
        self.paddingY = (0, 0, self._reversed_padding_repeated_twice[2], self._reversed_padding_repeated_twice[3])
        working = torch.nn.functional.pad(input, self.paddingX, mode=x_mode)
        working = torch.nn.functional.pad(working, self.paddingY, mode=y_mode)
        return torch.nn.functional.conv2d(working, weight, bias, self.stride, torch.nn.modules.utils._pair(0), self.dilation, self.groups)

    # Set padding mode
    x_mode = 'circular' if x_axis else 'constant'
    y_mode = 'circular' if y_axis else 'constant'

    targets = [pipeline.vae, pipeline.text_encoder, pipeline.unet]
    convolution_layers = []
    for target in targets:
        for module in target.modules():
            if isinstance(module, torch.nn.Conv2d):
                convolution_layers.append(module)

    for layer in convolution_layers:
        if isinstance(layer, LoRACompatibleConv) and layer.lora_layer is None:
            layer.lora_layer = lambda * x: 0

        layer._conv_forward = asymmetric_conv2d_convforward.__get__(layer, torch.nn.Conv2d)

    return pipeline

try:
    # 모델 로드 (모델을 GPU로 이동)
    pipeline = DiffusionPipeline.from_pretrained(
        model_name, 
        torch_dtype=torch.float16,  # float16 사용으로 GPU 메모리 효율화
        variant="fp16"  # 16-bit floating point 사용
    ).to('cuda')  # 모델을 GPU로 이동
    print("모델이 성공적으로 로드되었습니다.")
    
    pipeline = seamless_tiling(pipeline=pipeline, x_axis=True, y_axis=True)

    # 이미지 생성
    prompt = "seamless cat fabric textile pattern"
    negative_prompt = "low quality, blurry, deformed, bad anatomy"

    generator = torch.Generator(device='cuda').manual_seed(1)
    
    # pipeline을 통해 이미지 생성
    images = pipeline(
        prompt=prompt,  # 프롬프트는 문자열로 전달
        negative_prompt=negative_prompt,  # 부정 프롬프트도 문자열로 전달
        width=512,
        height=512,
        num_inference_steps=50,
        guidance_scale=7.5,  # 일반적으로 사용하는 값
        num_images_per_prompt=1,
        generator=generator  # GPU에서 생성된 seed 사용
    ).images

    # 생성된 이미지를 표시
    for i, image in enumerate(images):
        image.show(title=f"Generated Image {i+1}")

    # Reset seamless tiling
    seamless_tiling(pipeline=pipeline, x_axis=False, y_axis=False)
    torch.cuda.empty_cache()

except Exception as e:
    print(f"모델 로드 중 오류 발생: {e}")


Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00,  9.39it/s]


모델이 성공적으로 로드되었습니다.


100%|██████████| 50/50 [02:21<00:00,  2.83s/it]
